##Import Library and Dataset

In [2]:
# Install necessary libraries
!pip install scikit-learn matplotlib seaborn

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_samples

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/UTSClustering.csv', encoding='ISO-8859-1')

df.dropna(inplace=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Features Selection

In [4]:
# Remove constant features
constant_features = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=constant_features, inplace=True)

# Remove highly correlated features
numerical_df = df.select_dtypes(include=[np.number])
corr_matrix = numerical_df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
df.drop(columns=to_drop, inplace=True)

##Feature Engineering Standardization

In [5]:
# Select numerical columns only
numerical_df = df.select_dtypes(include=[np.number])
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numerical_df)

##Clustering Models

In [6]:
models = {
    'KMeans': KMeans(n_clusters=3, random_state=42),
    'Agglomerative Clustering': AgglomerativeClustering(n_clusters=3),
    'DBSCAN': DBSCAN(eps=0.5, min_samples=5),
    'Gaussian Mixture': GaussianMixture(n_components=3, random_state=42),
    'Spectral Clustering': SpectralClustering(n_clusters=3, affinity='nearest_neighbors', random_state=42)
}

##Fit The Models and Evaluate

In [ ]:
results = {}

for name, model in models.items():
    if name == 'DBSCAN':  # DBSCAN does not have a predict method
        model.fit(scaled_data)
        cluster_labels = model.labels_
    else:
        cluster_labels = model.fit_predict(scaled_data)

    # Some models may label all as noise (-1), skip if all labels same
    if len(set(cluster_labels)) > 1:
        silhouette = silhouette_score(scaled_data, cluster_labels)
        davies_bouldin = davies_bouldin_score(scaled_data, cluster_labels)
        calinski_harabasz = calinski_harabasz_score(scaled_data, cluster_labels)
    else:
        silhouette = davies_bouldin = calinski_harabasz = np.nan

    results[name] = {
        'Silhouette Score': silhouette,
        'Davies-Bouldin Index': davies_bouldin,
        'Calinski-Harabasz Score': calinski_harabasz,
        'Cluster Labels': cluster_labels
    }

##Visualize Evaluation Metrics

In [ ]:
metrics_df = pd.DataFrame(results).T
print(metrics_df)

##Choose The Best Model

In [ ]:
best_model_name = metrics_df['Silhouette Score'].idxmax()
best_model = models[best_model_name]
best_labels = results[best_model_name]['Cluster Labels']

##Visualize Clusters

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=numerical_df.iloc[:, 0], y=numerical_df.iloc[:, 1], hue=best_labels, palette='viridis', s=100)
plt.title(f"Cluster Scatter Plot - {best_model_name}")
plt.xlabel(numerical_df.columns[0])
plt.ylabel(numerical_df.columns[1])
plt.legend(title='Cluster')
plt.show()

##Silhouette Plot

In [ ]:
plt.figure(figsize=(8, 6))
silhouette_vals = silhouette_samples(scaled_data, best_labels)
sns.histplot(silhouette_vals, kde=True, bins=20, color="blue")
plt.title(f"Silhouette Plot - {best_model_name}")
plt.xlabel('Silhouette Score')
plt.ylabel('Frequency')
plt.show()

##Dendrogram

In [ ]:
if best_model_name == 'Agglomerative Clustering':
    plt.figure(figsize=(10, 6))
    linked = linkage(scaled_data, method='ward')
    dendrogram(linked)
    plt.title('Dendrogram - Agglomerative Clustering')
    plt.xlabel('Sample Index')
    plt.ylabel('Distance')
    plt.show()

##Final Conclusion

In [ ]:
best_model_description = {
    'KMeans': 'KMeans is a centroid-based clustering method that tries to minimize variance within clusters.',
    'Agglomerative Clustering': 'Agglomerative Clustering is a hierarchical clustering algorithm that builds a tree of clusters.',
    'DBSCAN': 'DBSCAN is a density-based algorithm that groups closely packed points and labels outliers as noise.',
    'Gaussian Mixture': 'Gaussian Mixture is a probabilistic model assuming data from several Gaussian distributions.',
    'Spectral Clustering': 'Spectral Clustering uses eigenvalues of a similarity matrix to reduce dimensions before clustering.'
}

print(f"Best Model: {best_model_name}")
print(best_model_description[best_model_name])

#**Tidak Dapat dieksekusi karena RAM penuh tetapi RAM masih banyak kosongnya**

#**Penjelasan**
**1. Pengumpulan dan Pembersihkan Data**
- Menggunakan Pandas untuk membaca data dan membersihkan

**2. Pengumpulan dan Pembersihkan Data**
- Melakukan seleksi fitur menggunakan teknik yang disebutkan
- Melakukan transformasi data seperti standardisasi atau normalisasi untuk meningkatkan performa model

**3. Pengumpulan dan Pembersihkan Data**
- Menjalankan berbagai Algoritma Clustering, seperti KMeans, AgglomerativeClustering, DBSCAN, GaussianMixture, dan SpectralClustering.

**4. Evaluasi**
- Menggunakan metrik evaluasi seperti Silhouette Score, Davies-Bouldin Index, dan Calinski-Harabasz Score.

**5. Visualisasi**
- Visualisasi dengan Silhouette Plot, Dendrogram, dan CLuster Scatter Plot

#**Analisis**

**1. Inkonsistensi antara Elbow Method dan Silhouette Score pada K-Means**\
Penyebab:
- Elbow Method mengidentifikasi jumlah cluster (K=5) dengan meminimalkan within-cluster sum of squares (WCSS), namun ini tidak selalu mencerminkan kualitas pemisahan cluster. Elbow method hanya mencari titik perubahan dalam penurunan WCSS tanpa mempertimbangkan bagaimana cluster tersebut terpisah dengan jelas.
- Silhouette Score yang rendah (0.3) menunjukkan bahwa meskipun cluster tersebut padat, kemungkinan tumpang tindih atau tidak cukup terpisah dengan baik.

Strategi Validasi Alternatif
- **Gap Statistic**\
Menghitung gap antara WCSS yang dihasilkan oleh model clustering terhadap distribusi acak untuk menemukan jumlah cluster yang optimal, dengan memperhatikan struktur data yang lebih kompleks.
- **Validasi Stabilitas via Bootstrapping**\
Menggunakan bootstrapping untuk menguji kestabilan cluster pada subset acak dari data.
- **Distribusi Non-Spherical**\
Jika data memiliki distribusi non-spherical, seperti elips atau klaster yang lebih kompleks, K-Means bisa gagal karena algoritma ini mengasumsikan cluster berbentuk bulat.

**2. Preprocessing untuk Fitur Numerik dan Kategorikal High-Cardinality**

**Metode Preprocessing untuk Fitur Numerik dan Kategorikal**
- **Fitur Numerik (Quantity, UnitPrice)**
Standardisasi atau normalisasi penting agar fitur numerik memiliki skala yang seragam, sehingga model tidak terpengaruh oleh perbedaan skala yang besar antara fitur-fitur tersebut.
- **Fitur Kategorikal High-Cardinality (Description)**\
Untuk menangani fitur teks dengan nilai kategori yang sangat banyak, metode encoding yang lebih canggih diperlukan.

**Risiko One-Hot Encoding**
- **High Cardinality**\
One-Hot Encoding mengubah fitur kategorikal menjadi vektor biner dengan dimensi yang sangat tinggi, yang menyebabkan masalah dalam komputasi dan dapat mengarah pada curse of dimensionality.

**Mengapa TF-IDF atau UMAP lebih baik?**
- TF-IDF dan UMAP lebih cocok karena mereka mengatasi masalah dimensionalitas tinggi dan memberi representasi yang lebih kompak dan bermakna untuk data kategorikal atau teks. Ini juga dapat lebih baik dalam menangkap pola dari data yang sangat terdistribusi.

**3. Menentukan Nilai Optimal Epsilon pada DBSCAN**

**Peran K-Distance Graph dan Kuartil Ke-3**
- K-Distance Graph digunakan untuk mencari nilai epsilon yang tepat untuk DBSCAN.
- Kuartil Ke-3 dari distribusi jarak ini bisa digunakan untuk memilih nilai epsilon secara adaptif.

**Menyesuaikan MinPts**
- MinPts (minimal points per cluster) harus disesuaikan berdasarkan kerapatan regional data. Di area dengan kepadatan tinggi, lebih sedikit titik (MinPts rendah) mungkin sudah cukup untuk membentuk cluster.

**4. Memperbaiki Pemisahan Cluster dengan Teknik Semi-Supervised**

**Mengatasi Overlap antara CLuster "High-Value Customers" dan "Bulk Buyers"**
- **Constrained Clustering (Semi-Supervised)**\
Teknik ini melibatkan pemberian beberapa batasan atau constraints pada algoritma clustering, seperti memberitahu algoritma bahwa dua titik tertentu harus berada dalam cluster yang sama (must-link) atau berbeda (cannot-link).
- **Integrasi Metric Learning (Mahalanobis Distance)**\
Mahalanobis distance lebih robust dalam mengukur jarak antara titik-titik yang berbeda distribusi, dengan mempertimbangkan variansi dan korelasi antar fitur.

**Tantangan Interpretabilitas Bisnis**
- Pendekatan non-Euclidean seperti Mahalanobis distance dan constrained clustering bisa memperumit interpretasi bisnis, karena jarak antar titik tidak lagi berbasis pada ruang Euclidean tradisional.

**5. Desain Fitur Temporal dan Risiko Data Leakage**

**Desain Fitur Temporal dari InvoiceDate**
- **Hari dalam Minggu dan Jam Pembelian**\
Fitur seperti day of the week atau hour of purchase bisa digunakan untuk mengidentifikasi pola pembelian periodik, seperti pembelian lebih banyak di pagi hari atau malam hari.

**Risiko Data Leakage dengan Agregasi Temporal**
- **Data Leakage**\
Jika kita menggunakan rata-rata pembelian bulanan tanpa melakukan time-based cross-validation, kita bisa memasukkan informasi masa depan ke dalam model, yang mengarah pada data leakage.
- **Lag Features**\
Lag features misalnya, pembelian 7 hari sebelumnya dapat memperkenalkan noise karena mereka mengandalkan data masa lalu yang tidak relevan, atau pola musiman yang tidak konsisten.